# Paso 10. Ecuación de Poisson bidimensional

$$
\frac{\partial^2 p}{\partial x^2} + \frac{\partial^2 p}{\partial y^2} = b
$$

#### Condiciones iniciales y valores de frontera

Para el dominio $[0, 2]\times[0, 1]$

\begin{eqnarray}
p(x,y) &=& 0\\
p(0,y) = p(2,y) &=& 0\\
p(x,0) = p(x,1) &=& 0\\
\end{eqnarray}

En esta ocación contamos con fuentes que vienen a representar la no homogeneidad de la ecuación. Para las fuentes su perfil inicial es:

\begin{eqnarray}
b(i,j) &=& 0\\
b\Big(\frac{nx}{4}, \frac{ny}{4}\Big) &=& 100\\
b\Big(\frac{3}{4}nx, \frac{3}{4}ny\Big) &=& -100\\
\end{eqnarray}


#### Discretización
$$
\frac{p(i+1,j)_n - 2p(i,j)_n + p(i-1,j)_n}{\Delta x^2} + \frac{p(i,j+1)_n - 2p(i,j)_n + p(i,j-1)_n}{\Delta y^2} = b
$$

De manera similar a la ecuación de Laplace, solucionando para $p(i,j)_n$, se obtiene

$$
p(i,j)_n = \frac{ \Delta y^2\Big[ p(i+1,j)_n + p(i-1,j)_n \Big] + \Delta x^2\Big[ p(i,j+1)_n + p(i,j-1)_n \Big] - b(i,j)_n \Delta x^2 \Delta y^2 } {2(\Delta x^2 + \Delta y^2)}
$$

Y con esta ecuación realizamos un proceso iterativo hasta encontrar el profile de estabilidad del sistema

In [14]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

def update(i, profile, source):
    #store the actual state of the array
    p_n = p;

    pn_i_j   = p_n[1:,1:];
    pn_im1_j = np.roll(p,  1, axis=1)[1:, 1:];
    pn_ip1_j = np.roll(p, -1, axis=1)[1:, 1:];
    pn_i_jm1 = np.roll(p,  1, axis=0)[1:, 1:];
    pn_i_jp1 = np.roll(p, -1, axis=0)[1:, 1:];
    bn_ij    = source[1:,1:];
    
    p[1:,1:] = ((dy**2)*(pn_ip1_j + pn_im1_j) + (dx**2)*(pn_i_jp1 + pn_i_jm1) - bn_ij*(dx**2)*(dy**2)) / (2*(dx**2 + dy**2))

    """
    p = 0 at x = 0 & x = 2
    p = 0 at y = 0 & y = 1
    """
    p[:,0] = 0;
    p[:,-1] = 0;
    p[0,:] = 0;
    p[-1,:] = 0;

    ax.clear();
    ax.set_zlim(-0.33,0.33)
    ax.grid(False);

    profile = ax.plot_surface(X, Y, p, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0, antialiased=False);
    
    return profile,


# How much we want to advance in time?
# at 30 fps if we want simulate 10 seconds => 30*10
t_seconds = 30 * 10;

# The time step size 
dt = 0.005;

partitions = 31;

data_x = np.linspace(0.0, 2.0, num=partitions, retstep=True);
x  = data_x[0]
dx = data_x[1];

data_y = np.linspace(0.0, 1.0, num=partitions, retstep=True);
y  = data_x[0]
dy = data_x[1];

X, Y = np.meshgrid(x, y);

#initial conditions
p = np.zeros((partitions, partitions));
"""
p = 0 at x = 0 & x = 2
p = 0 at y = 0 & y = 1
"""
p[:,0]  = 0;
p[:,-1] = 0;
p[0,:]  = 0;
p[-1,:] = 0;
    
"""
Sources (Spikes)
"""
source = np.zeros((partitions, partitions));
source[partitions/4, partitions/4] = 100;
source[partitions*3/4, partitions*3/4] = -100;


# Create a figure and a 3D Axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d');
ax.set_xlim(0,2)
ax.set_ylim(0,1)
ax.set_zlim(-0.33,0.33)
ax.grid(False);
ax.view_init(azim=-150)
profile = ax.plot_surface(X, Y, p, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0, antialiased=False);

#Animate
anim = animation.FuncAnimation(fig, update, fargs=(profile,source), frames=t_seconds, blit=False)
anim.save('2D_Poisson_Equation_00.mp4', fps=30, writer="ffmpeg", codec="libx264");
plt.close(fig);

In [15]:
# You must have configured ffmpeg in your machine to run the code below
from IPython.display import HTML
video = open("2D_Poisson_Equation_00.mp4", "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(video_tag)